In [1]:
import json
import time
from functools import reduce

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm.autonotebook import tqdm

from py2neo import Graph, Node, Relationship

In [2]:
public_address = '54.174.175.98'
graph = Graph('bolt://{}:7687'.format(public_address), auth=('neo4j','myneo'))

def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False, verbose=True):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    if verbose:
        print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df

In [ ]:
CALL apoc.periodic.iterate("
MATCH (q:Quanta)-[:PUBLISHED_IN]->(y:Year)
RETURN [q,y] AS ns
","
WITH head(ns) AS q, last(ns) AS y
MATCH (z:Year) WHERE z.year >= y.year
MERGE (q)-[m:METRICS_IN]->(z)
SET m.citations = size((z)<-[:PUBLISHED_IN]-(:Quanta)-[:CITES]->(q))",
{batchSize:10000, iterateList:true, parallel:false});